---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [27]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [28]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [30]:
def degree_distribution(G):
    degrees = G.degree()
    degree_values = sorted(set(degrees.values()))
    histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes( G)) for i in degree_values]
    return histogram

def graph_identification():
    methods = []
    for G in P1_Graphs:
        clustering = nx.average_clustering(G)
        shortest_path = nx.average_shortest_path_length(G)
        degree_hist = degree_distribution(G)
        if len(degree_hist)>10:
            methods.append('PA')
        elif clustering < 0.1:
            methods.append('SW_H')
        else:
            methods.append('SW_L')
    return methods

graph_identification()


['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [31]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [32]:
def salary_predictions():
    #Making the dataframe
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    
    #making the X_test and y_test
    df_test = df[df['ManagementSalary'].isnull()]
    X_test = df_test[['clustering', 'degree']]
    y_test = df_test[[]]
    
    #making the X_train and y_train
    df_train = df.dropna()
    X_train = df_train[['clustering', 'degree']]
    y_train = df_train['ManagementSalary']
    
    #making the test and predicting the results
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.linear_model import LogisticRegression
    lr = LogisticRegression().fit(X_train, y_train)
    s_tr = lr.score(X_train, y_train)
    predict_proba = lr.predict_proba(X_test)[:,1]
    
    
    return pd.Series(predict_proba, y_test.index)

salary_predictions()

1       0.174804
2       0.573707
5       0.958913
8       0.133046
14      0.385540
18      0.209978
27      0.284991
30      0.341991
31      0.199421
34      0.141151
37      0.129840
40      0.257518
45      0.176206
54      0.238868
55      0.187842
60      0.231781
62      0.993961
65      0.480337
77      0.083321
79      0.143355
97      0.100571
101     0.081153
103     0.210580
108     0.136683
113     0.490525
122     0.077152
141     0.433465
142     0.893342
144     0.069338
145     0.369209
          ...   
913     0.068323
914     0.091908
915     0.015925
918     0.128170
923     0.057262
926     0.084503
931     0.069352
934     0.031020
939     0.029790
944     0.015925
945     0.055050
947     0.111331
950     0.123286
951     0.061128
953     0.049515
959     0.015925
962     0.016592
963     0.146885
968     0.088860
969     0.082330
974     0.074420
984     0.029790
987     0.089246
989     0.088860
991     0.088860
992     0.017287
994     0.015925
996     0.0159

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [33]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [34]:
def new_connections_predictions():
    future_connections['preferential attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    future_connections['Common Neighbors'] = future_connections.index.map(lambda x: len(list(nx.common_neighbors(G, x[0], x[1]))))
    
    #making the training set
    train_set = future_connections.dropna()
    X_train = train_set[['preferential attachment', 'Common Neighbors']]
    y_train= train_set['Future Connection']
    
    #making the test set
    test_set = future_connections[future_connections['Future Connection'].isnull()]
    X_test = test_set[['preferential attachment', 'Common Neighbors']]
    y_test = test_set[[]]
    
    
    
    #traing the set
    from sklearn.linear_model import LogisticRegression
    lr = LogisticRegression().fit(X_train, y_train)
    s_tr = lr.score(X_train, y_train)
    predict_proba = lr.predict_proba(X_test)[:,1]
    
    return pd.Series(predict_proba, y_test.index)

new_connections_predictions()

(107, 348)    0.038540
(542, 751)    0.014637
(20, 426)     0.641230
(50, 989)     0.014916
(942, 986)    0.015221
(324, 857)    0.014877
(13, 710)     0.164049
(19, 271)     0.108621
(319, 878)    0.015014
(659, 707)    0.014666
(49, 843)     0.015102
(208, 893)    0.014733
(377, 469)    0.006421
(405, 999)    0.026935
(129, 740)    0.022686
(292, 618)    0.023325
(239, 689)    0.015033
(359, 373)    0.009146
(53, 523)     0.040090
(276, 984)    0.015087
(202, 997)    0.015147
(604, 619)    0.049954
(270, 911)    0.015029
(261, 481)    0.080042
(200, 450)    0.856947
(213, 634)    0.014551
(644, 735)    0.049097
(346, 553)    0.013875
(521, 738)    0.012638
(422, 953)    0.022221
                ...   
(672, 848)    0.015029
(28, 127)     0.972458
(202, 661)    0.013762
(54, 195)     0.999926
(295, 864)    0.014834
(814, 936)    0.014594
(839, 874)    0.015221
(139, 843)    0.014858
(461, 544)    0.011822
(68, 487)     0.011684
(622, 932)    0.014766
(504, 936)    0.019094
(479, 528) 